In [3]:
#!/usr/bin/env python
import numpy as np
import pandas as pd
import time
import csv
import re
import json
import xlrd
import os
from datetime import datetime, date, timedelta
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
#from flatten_json import flatten
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
import time
import copy
from fuzzywuzzy import process, fuzz


def load_n_explode(file_res="API_results_" + time.strftime("%Y%m%d") + ".csv"):
    # file_res = "API_results_20210823.csv"
    my_df = pd.read_csv(file_res, error_bad_lines=False)

    ## Categories
    # my_df["Categories"]
    # Willnjust keep 2 levels.
    my_df["Categories"] = my_df["Categories"].map(eval, na_action='ignore')
    new_df = my_df["Categories"].apply(pd.Series)
    my_df["Categories"] = new_df[0].apply(pd.Series).UrlFriendlyName
    my_df["Sub_Categories"] = new_df[1].apply(pd.Series).UrlFriendlyName


    ###********NEW> LIMIT TO WINES ONLY
    my_df = my_df[my_df['Categories'].isin(['red-wine', 'white-wine'])]

    ## WORK AROUND> NOT SURE WHY. TODO
    my_df = my_df[my_df['Stockcode'] != 'ER_2000003422_RX2386']
    my_df = my_df[my_df['Stockcode'] != 'ER_1000004375_CALSG16']

    

    ## Reviews
    my_df["Reviews"] = my_df["Reviews"].map(eval, na_action='ignore')
    # Try with first 2 reviews
    new_df = my_df["Reviews"].apply(pd.Series)
    # First
    my_df["Review1_auth"] = new_df[0].apply(pd.Series).author.apply(pd.Series).Value
    my_df["Review1_authorcontent"] = new_df[0].apply(pd.Series).authorcontent.apply(pd.Series).Value
    my_df["Review1_points"] = new_df[0].apply(pd.Series).points.apply(pd.Series).Value
    my_df["Review1_source"] = new_df[0].apply(pd.Series).source.apply(pd.Series).Value
    my_df["Review1_text"] = new_df[0].apply(pd.Series).text.apply(pd.Series).Value
    my_df["Review1_vintage"] = new_df[0].apply(pd.Series).vintage.apply(pd.Series).Value
    # Second
    my_df["Review2_auth"] = new_df[1].apply(pd.Series).author.apply(pd.Series).Value
    my_df["Review2_authorcontent"] = new_df[1].apply(pd.Series).authorcontent.apply(pd.Series).Value
    my_df["Review2_points"] = new_df[1].apply(pd.Series).points.apply(pd.Series).Value
    my_df["Review2_source"] = new_df[1].apply(pd.Series).source.apply(pd.Series).Value
    my_df["Review2_text"] = new_df[1].apply(pd.Series).text.apply(pd.Series).Value
    my_df["Review2_vintage"] = new_df[1].apply(pd.Series).vintage.apply(pd.Series).Value

    # Illl make a deep copy for later
    full_df = copy.deepcopy(my_df)
    # full_df = full_df

    # Additional details
    my_df["AdditionalDetails"] = my_df["AdditionalDetails"].map(eval, na_action='ignore')
    # Can't use nested lists of JSON objects in pd.json_normalize
    my_df = my_df.explode(column="AdditionalDetails").reset_index(drop=True)

    # Hacky, but it works... so we wont be touching this stuff!
    add_df = pd.DataFrame(pd.json_normalize(my_df["AdditionalDetails"]))
    del add_df["DisplayName"]
    df = pd.concat([my_df, add_df], axis=1)
    df = df.pivot(index='Stockcode', columns='Name', values='Value').reset_index().drop_duplicates(subset=['Stockcode'],
                                                                                                   keep=False)

    # Check point, and also a way to get rid of headers
    newdf = pd.merge(full_df, df, on='Stockcode')
    newdf["Mystery"] = newdf["Description"].str.lower().str.contains("wraps")
    # This is an old secret seleciton one. Only two, so will drop them
    newdf = newdf[~newdf["Description"].str.contains("Secret Selection")]
    newdf = newdf[~newdf["Stockcode"].str.contains("672366")]
    # newdf = newdf.drop_duplicates(subset=['Stockcode'], keep=False)
    return newdf



def giveaway(df):
    gives = df.copy(deep=True)
    gives = gives[['Stockcode','Description','webproductname','Prices.singleprice.Value','Prices.promoprice.Value','Prices.promoprice.BeforePromotion','Prices.promoprice.AfterPromotion','IsForDelivery']]
    gives = gives[gives.webproductname.notnull()]
    gives = gives[gives.webproductname.notna()]
    gives = gives[gives["Description"].str.lower().str.contains("wraps")]
    gives = gives[~gives["webproductname"].str.lower().str.contains("wraps")]
    gives = gives[gives["IsForDelivery"]]
    gives["METHOD"] = "giveaway"
    return gives



def ohe(df):
    #### ONE HOT ENCODED
    ##### First I split into numeric and nominal. OHE the nominal
    known=df
    exclude_col = known.select_dtypes(include=np.number).columns.tolist() + ["Stockcode"]
    my_df_num = known[exclude_col]
    my_df_cat = known.drop(exclude_col, axis=1)
    # my_df_cat.to_csv("FIN.csv")
    my_df_cat_ohe = pd.get_dummies(my_df_cat)
    my_df_ohe = pd.concat([my_df_num, my_df_cat_ohe], axis=1)
    my_df_ohe = my_df_ohe.fillna(0)
    my_df_ohe = my_df_ohe.replace(np.nan, 0)

    # Drop duplicates #TODO check whats better to keep
    my_df_ohe = my_df_ohe.loc[:, ~my_df_ohe.columns.duplicated()]
    # Clean up names
    my_df_ohe.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in
                         my_df_ohe.columns.values]

    return my_df_ohe

def get_knn(df, thresh):

    myst  = df[df["Mystery"]]
    known  = df[~df["Mystery"]]

    myst_nn = myst.drop("Stockcode", axis=1)
    myst_nn = myst_nn.drop("Mystery", axis=1)
    known_nn = known.drop("Stockcode", axis=1)
    known_nn = known_nn.drop("Mystery", axis=1)

    # Create the k-NN model using k=5
    nn_abs = NearestNeighbors(n_neighbors=1, algorithm='auto')

    # Fit it
    nn_abs.fit(known_nn)

    results_wine = []

    for index in range(len(myst.index)):
        distance, matches = nn_abs.kneighbors(myst_nn.iloc[[index]], 1, return_distance=True)
        results_wine.append(
            {
                #'Mystery': "https://www.danmurphys.com.au/product/" + str(myst['Stockcode'].iloc[[index][0]]),
                'Stockcode_x': str(myst['Stockcode'].iloc[[index][0]]),
                #'Matched': "https://www.danmurphys.com.au/product/" + str(known["Stockcode"].iloc[matches[0][0]]),
                'Stockcode_y': str(known["Stockcode"].iloc[matches[0][0]]),
                'Distance': str(distance[0][0])

            }
        )

    matched = pd.DataFrame(results_wine)
    matched["MatchLevel"] = np.where(matched['Distance'].astype(float) < float(thresh), "Good", "Poor")
    #matched = matched.sort_values(['MatchLevel', 'Savings'], ascending=[True, False])
    matched["METHOD"] = "KNN"
    return matched


def lazy_desc(df, keep1):
    #df = wide
    #keep1 = keep_nlp
    kept = df[keep1]

    kept.reset_index(drop=True, inplace=True)
    myst = kept[kept["Mystery"]]
    known = kept[~kept["Mystery"]]
    known = known[known["Review1_text"] != "[...]"]
    known = known[known["Review2_text"] != "[...]"]
    
    known = known[known["Review1_text"] != "[..]"]
    known = known[known["Review2_text"] != "[..]"]
    
    known = known[known["Review1_text"] != "[....]"]
    known = known[known["Review2_text"] != "[....]"]
 
    
    # Description match
    desc_match = pd.merge(myst[myst['RichDescription'].notna()], known, on=['RichDescription'], how='inner')
    desc_match["METHOD"] = "desc_match"
    # Web match
    webdesc_match = pd.merge(myst[myst['webdescriptionshort'].notna()], known, on=['webdescriptionshort'], how='inner')
    webdesc_match["METHOD"] = "webdesc_match"
    # Review match (TODO more than 2 deep)
    rev_match = pd.merge(myst[myst['Review1_text'].notna()], known, on=['Review1_text'], how='inner')
    rev_match["METHOD"] = "rev_match"
    #rev_match = rev_match[~[rev_match['Review1_text'] == [...]]]
    rev1_match = pd.merge(myst[myst['Review2_text'].notna()], known, on=['Review2_text'], how='inner')
    rev1_match["METHOD"] = "rev2_match" # I know. Using normal index for people...
    #rev_match = rev_match[~[rev_match.['Review1_text'].str.contains("[...]")]]
    text_match = desc_match.append(webdesc_match).append(rev_match).append(rev1_match).reset_index()
    text_match = text_match[['Stockcode_x', 'Stockcode_y',"METHOD"]]
    text_match["MatchLevel"] = "Good"
    return text_match


## Try Fuzzywuzzy

def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

#fuzzy_merge(myst, known, 'webdescriptionshort', 'webdescriptionshort', threshold=80)


def make_clickable(val):
    # target _blank to open new window
    val1 = 'https://www.danmurphys.com.au/product/' + str(val)
    return '<a target="_blank" href="{}">{}</a>'.format(val1, val)


## CONFIG

keep_nlptdf =['Stockcode',
 'webdescriptionshort',
'RichDescription']

keep_ohe =['Categories',
 'Mystery',
 'Stockcode',
 'PackageSize',
 #'Prices.inanysixprice.Message',
 'Prices.inanysixprice.Value',
 'Sub_Categories',
 'Review1_auth',
 'Review1_points',
 'Review1_source',
 'awardwinner',
 'glutenfree',
 'preservativefree',
 'varietal',
 'webalcoholpercentage',
 'webbottleclosure',
 'webcountryoforigin',
 'webfoodmatch',
 'webisorganic',
 'webisvegan',
 'webliquorsize',
 'webmaincategory',
 'webregionoforigin',
 'webstateoforigin',
 #'webtotalreviewcount',
 'webwinebody',
 'webwinestyle',
 'IsForDelivery']


keep_nlp =['Categories',
 'Stockcode',
 'Mystery',
 'PackageSize',
 'RichDescription',
 'Review1_text',
 'Review2_text',
 'Prices.inanysixprice.Message',
 'Prices.inanysixprice.Value',
 'Sub_Categories',
 'Review1_auth',
 'Review1_points',
 'Review1_source',
 'awardwinner',
 'glutenfree',
 'preservativefree',
 'varietal',
 'webalcoholpercentage',
 'webbottleclosure',
 'webcountryoforigin',
 'webdescriptionshort',
 'webfoodmatch',
 'webisorganic',
 'webisvegan',
 'webliquorsize',
 'webmaincategory',
 'webregionoforigin',
 'webstateoforigin',
 'webtotalreviewcount',
 'webwinebody',
 'webwinestyle',
 'IsForDelivery']


In [23]:
keep_ohe_xg =['Categories',
 'Mystery',
 'Stockcode',
#'PackageSize',
 #'Prices.inanysixprice.Message',
 'Prices.inanysixprice.Value',
 'Sub_Categories',
 #'Review1_auth',
 #'Review1_points',
 #'Review1_source',
 #'awardwinner',
 'glutenfree',
 'preservativefree',
# 'varietal',
# 'webalcoholpercentage',
 'webbottleclosure',
 #'webcountryoforigin',
 #'webfoodmatch',
 #'webisorganic',
 #'webisvegan',
 'webliquorsize',
 #'webmaincategory',
 'webregionoforigin',
 'webstateoforigin',
 #'webtotalreviewcount',
 #'webwinebody',
 #'webwinestyle',
 'IsForDelivery']

In [5]:
#### RUNNING HERE

#input_file = "API_results_" + time.strftime("%Y%m%d") + ".csv"
input_file = "API_results_20210906.csv"
wide = load_n_explode(input_file)



/home/stu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (101,106) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [165]:
mystery = wide[wide["Mystery"]]

In [166]:
mystery.to_csv("Stockocdes.csv", mode='w', columns=['Stockcode'], index=False)

In [24]:
wide = wide[wide['Stockcode'] != 'ER_2000003439_MTRESSHZ']

In [162]:
## Get the easy matches
gives = giveaway(wide)
lazy = lazy_desc(wide,keep_nlp)

#now the OHE and KNN
ohe_file = ohe(wide[keep_ohe])
knns = get_knn(ohe_file,1.5)

# Cleanup and save it
#matches = knns.append(lazy)
matches = lazy


In [163]:
# Check if its in stock
matches = matches[matches.MatchLevel.str.contains("Good")] # Get ride of bad matches first
avail = wide[wide["IsForDelivery"]][["Stockcode","Mystery",'varietal','webbottleclosure','Prices.singleprice.Value','Prices.promoprice.Value','Prices.promoprice.BeforePromotion','Prices.promoprice.AfterPromotion']]
matches = pd.merge(avail,matches, left_on='Stockcode', right_on="Stockcode_x",how='left')


In [157]:
matches["Savings"] = matches['Prices.promoprice.BeforePromotion'] - matches['Prices.promoprice.AfterPromotion']

,Stockcode,Mystery,varietal,Prices.singleprice.Value,Prices.promoprice.Value,Prices.promoprice.BeforePromotion,Prices.promoprice.AfterPromotion,Stockcode_x,Stockcode_y,METHOD,MatchLevel,Savings
0,133780,False,Cabernet Sauvignon,62.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,334645,False,Moscato,4.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,742755,False,Pinot Noir,20.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,789896,False,Pinot Noir,24.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,901808,False,Verdelho,11.95,10.0,11.95,10.0,NaN,NaN,NaN,NaN,1.95
...,...,...,...,...,...,...,...,...,...,...,...,...
7816,73151,False,Chardonnay,11.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7817,903952,False,Pinot Noir,21.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7818,20345,False,Shiraz,12.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7819,904232,False,Shiraz,31.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
matches = pd.merge(matches, wide[["Description","Stockcode"]], left_on='Stockcode_y', right_on="Stockcode",how='left')


,Stockcode_x,Mystery,varietal,Prices.singleprice.Value,Prices.promoprice.Value,Prices.promoprice.BeforePromotion,Prices.promoprice.AfterPromotion,Stockcode_x,Stockcode_y,METHOD,MatchLevel,Savings,Description,Stockcode_y
0,133780,False,Cabernet Sauvignon,62.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,334645,False,Moscato,4.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,742755,False,Pinot Noir,20.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,789896,False,Pinot Noir,24.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,901808,False,Verdelho,11.95,10.0,11.95,10.0,NaN,NaN,NaN,NaN,1.95,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7816,73151,False,Chardonnay,11.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7817,903952,False,Pinot Noir,21.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7818,20345,False,Shiraz,12.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7819,904232,False,Shiraz,31.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
#matches.columns = ['Stockcode_x', 'Stockcode_y', 'Distance', 'MatchLevel', 'Stockcode_x', 'varietal', 'Prices.singleprice.Value', 'Prices.promoprice.Value', 'Prices.promoprice.BeforePromotion', 'Prices.promoprice.AfterPromotion', 'Savings', 'Description', 'Stockcode_Z']

matches = matches[['Stockcode_x', 'Stockcode_y', 'Description', 'varietal','webbottleclosure','Prices.promoprice.BeforePromotion', 'Prices.promoprice.AfterPromotion', 'Savings']]
matches = matches.sort_values(['Savings'], ascending=[False])

matches = matches.loc[:,~matches.columns.duplicated()]

matches = matches.drop_duplicates()

In [161]:
matches

,Stockcode_x,Stockcode_y,Description,varietal,Prices.promoprice.BeforePromotion,Prices.promoprice.AfterPromotion,Savings
3935,ER_1000003887_UW-194022,NaN,NaN,Shiraz,1410.00,810.0,600.00
4207,ER_2000003549_UWVINBVSHIRAZ18,NaN,NaN,Shiraz,479.94,210.0,269.94
1926,ER_1000004523_P01336,NaN,NaN,Shiraz,604.14,360.0,244.14
2919,787635,NaN,NaN,Shiraz,934.99,700.0,234.99
1611,B999999000966,NaN,NaN,Mixed Reds,383.94,149.9,234.04
...,...,...,...,...,...,...,...
7816,73151,NaN,NaN,Chardonnay,NaN,NaN,NaN
7817,903952,NaN,NaN,Pinot Noir,NaN,NaN,NaN
7818,20345,NaN,NaN,Shiraz,NaN,NaN,NaN
7819,904232,NaN,NaN,Shiraz,NaN,NaN,NaN


In [146]:
#pd.merge(wide[wide["Mystery"]][["Stockcode","Description","varietal"]],matches, how='left',left_on='Stockcode', right_on="Stockcode_x")

In [147]:

#csv_file = "Match_results" + time.strftime("%Y%m%d") + ".csv"
#matches.to_csv(csv_file)

#from datetime import date, timedelta  
#yesterday = date.today() - timedelta(days=1) 
#today = date.today()   
#file_yes = "Match_results" + yesterday.strftime("%Y%m%d") + ".csv" 
#file_tod = "Match_results" + today.strftime("%Y%m%d") + ".csv"
#df1 = pd.read_csv(file_yes).iloc[:, 1:]
#df2 = pd.read_csv(file_tod).iloc[:, 1:]
#df_diff = pd.concat([df1,df2]).drop_duplicates(keep=False)



In [160]:
#matches = df_diff
final = matches.style.format({'Stockcode_x': make_clickable, 'Stockcode_y': make_clickable, }) \
    .bar(subset=['Savings'], align='mid', color=['#5fba7d']) \
    .bar(subset=['Savings'], align='mid', color=['#5fba7d']) \
    .hide_index()


#writing HTML Content
heading = '<h1> Matched wines</h1>'
subheading = '<h3> Results sub header </h3>'
# Using .now() from datetime library to add Time stamp
now = datetime.now()
current_time = now.strftime("%m/%d/%Y %H:%M:%S")
header = '<div class="top">' + heading + subheading +'</div>'
footer = '<div class="bottom"> <h3> This Report has been Generated on'+ current_time +'</h3> </div>'
content = final
# Concating everything to a single string

html = header + content.render() + footer
html_file = "Match_new.html"
with open(html_file,'w+') as file:
    file.write(html)


In [16]:
cork = wide[wide["Mystery"]]

In [30]:
cork = cork[cork["webbottleclosure"] == "Cork"]

In [31]:
cork = pd.merge(cork, avail, left_on='Stockcode', right_on="Stockcode")

In [33]:
cork[['Stockcode','Description', 'varietal', 'Prices.promoprice.BeforePromotion', 'Prices.promoprice.AfterPromotion']]

In [34]:
## match on 
variety
Cost per bottle
closure
standard drinks
alcohol %
region
size

Once matched, then if the count is greater than 1



In [7]:
import xgboost as xgb

from sklearn import datasets

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder 

from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score

In [58]:
ohe_file = copy.deepcopy(wide[keep_ohe])

In [59]:
#ohe_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22530 entries, 0 to 22603
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Categories                  22530 non-null  object 
 1   Mystery                     22530 non-null  bool   
 2   Stockcode                   22530 non-null  object 
 3   PackageSize                 22111 non-null  object 
 4   Prices.inanysixprice.Value  15056 non-null  float64
 5   Sub_Categories              22227 non-null  object 
 6   Review1_auth                6748 non-null   object 
 7   Review1_points              6749 non-null   object 
 8   Review1_source              3393 non-null   object 
 9   awardwinner                 2190 non-null   object 
 10  glutenfree                  878 non-null    object 
 11  preservativefree            172 non-null    object 
 12  varietal                    21610 non-null  object 
 13  webalcoholpercentage        218

In [10]:
#ohe_file["Sub_Categories"].drop_duplicates()

In [60]:
ohe_file = ohe_file[ohe_file["Sub_Categories"] == "shiraz"]

In [61]:
ohe_file["webliquorsize"] = ohe_file["webliquorsize"].str.extract(r'(^[0-9]{1,4})').fillna(0)

In [62]:
ohe_file["webliquorsize"] = pd.to_numeric(ohe_file["webliquorsize"])

In [63]:
ohe_file["glutenfree"].fillna(0, inplace=True)
ohe_file["preservativefree"].fillna(0, inplace=True)
ohe_file["Prices.inanysixprice.Value"].fillna(0, inplace=True)

In [64]:
ohe_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4330 entries, 2 to 22602
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Categories                  4330 non-null   object 
 1   Mystery                     4330 non-null   bool   
 2   Stockcode                   4330 non-null   object 
 3   PackageSize                 4224 non-null   object 
 4   Prices.inanysixprice.Value  4330 non-null   float64
 5   Sub_Categories              4330 non-null   object 
 6   Review1_auth                1771 non-null   object 
 7   Review1_points              1764 non-null   object 
 8   Review1_source              879 non-null    object 
 9   awardwinner                 544 non-null    object 
 10  glutenfree                  4330 non-null   object 
 11  preservativefree            4330 non-null   object 
 12  varietal                    4194 non-null   object 
 13  webalcoholpercentage        4186

In [65]:

ohe_file = ohe(ohe_file)
ohe_file = ohe_file.reset_index(drop=True)

In [66]:
ohe_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4330 entries, 0 to 4329
Columns: 1248 entries, Prices.inanysixprice.Value to webwinestyle_Sweet
dtypes: bool(2), float64(1), int64(1), object(1), uint8(1243)
memory usage: 5.2+ MB


In [90]:
myst  = ohe_file[ohe_file["Mystery"]]
known  = ohe_file[~ohe_file["Mystery"]]

myst_nn = myst.drop("Stockcode", axis=1)
myst_nn = myst_nn.drop("Mystery", axis=1)
known_nn = known.drop("Stockcode", axis=1)
known_nn = known_nn.drop("Mystery", axis=1)

In [68]:
#list(ohe_file)

In [95]:
X = known_nn
y = known["Stockcode"]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=5)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X, y).predict(X)


In [96]:
print("Number of mislabeled points out of a total %d points : %d"
       % (X.shape[0], (y != y_pred).sum()))


Number of mislabeled points out of a total 4113 points : 509


In [97]:
myst["Predicted_NB"] = gnb.predict(myst_nn)

/home/stu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [98]:
myst[["Stockcode","Predicted_NB"]]

,Stockcode,Predicted_NB
324,ER_1000004893_CDESHI,87915
475,MYSTERY32,413028
481,MYSTERY41,431185
510,MYSTERY119,143790
511,MYSTERY12,379631
...,...,...
4046,MYSTERY36,740515
4071,MYSTERY132,574764
4072,MYSTERY142,715320
4073,MYSTERY86,916488


In [99]:
gnb.score(X,y)

0.8762460491125699

In [86]:
myst  = wide[wide["Mystery"]]
known  = wide[~wide["Mystery"]]

myst_nn = myst.drop("Stockcode", axis=1)
myst_nn = myst_nn.drop("Mystery", axis=1)
known_nn = known.drop("Stockcode", axis=1)
known_nn = known_nn.drop("Mystery", axis=1)

X=known_nn
y=known["Stockcode"]
X.

In [88]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
encoder = OrdinalEncoder()
data_encoded = encoder.fit_transform(X)

TypeError: argument must be a string or number

In [83]:
from sklearn.naive_bayes import CategoricalNB
clf = CategoricalNB()
clf.fit(X, y)

ValueError: could not convert string to float: 'red-wine'

Number of mislabeled points out of a total 4113 points : 4021


In [21]:



model = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=6,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 early_stopping_rounds = 10,
# nthread=10,
 #scale_pos_weight=1,
 seed=27)


In [22]:
model.fit(X_train, y_train)

/home/stu/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:49:53] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:541: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:49:55] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

In [25]:
y_pred = model.predict(X_test) 


In [27]:

predictions = [value for value in y_pred]

accuracy = accuracy_score(y_test, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 0.00%


In [28]:
y_pred

array(['907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '907310',
       '907310', '907310', '907310', '907310', '907310', '9073

Hi
